In [30]:
import numpy as np
import math
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import time
import networkx as nx
from cspy import BiDirectional
import os


# Read data

In [31]:


def preprocess_cvrp(data_file):
    # Step 1: Read and parse the data
    with open(data_file, "r") as file:
        lines = file.readlines()
    
    # Extract sections
    node_coord_section = []
    demand_section = []
    capacity = 0
    depot = 0
    
    current_section = None
    for line in lines:
        line = line.strip()
        if line.startswith("NODE_COORD_SECTION"):
            current_section = "NODE_COORD"
        elif line.startswith("DEMAND_SECTION"):
            current_section = "DEMAND"
        elif line.startswith("DEPOT_SECTION"):
            current_section = "DEPOT"
        elif line.startswith("CAPACITY"):
            capacity = int(line.split(":")[1])
        elif current_section == "NODE_COORD" and line and not line.startswith("EOF"):
            parts = list(map(int, line.split()))
            node_coord_section.append((parts[0], parts[1], parts[2]))
        elif current_section == "DEMAND" and line and not line.startswith("EOF"):
            parts = list(map(int, line.split()))
            demand_section.append((parts[0], parts[1]))
        elif current_section == "DEPOT" and line and not line.startswith("EOF"):
            if depot == 1:
                continue
            depot = int(line)
            # print(depot)

    # Convert to pandas DataFrames
    nodes_df = pd.DataFrame(node_coord_section, columns=["Node", "X", "Y"])
    demands_df = pd.DataFrame(demand_section, columns=["Node", "Demand"])

    # Step 2: Compute distance matrix
    num_nodes = len(nodes_df)
    distance_matrix = np.zeros((num_nodes, num_nodes))

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                x1, y1 = nodes_df.loc[i, ["X", "Y"]]
                x2, y2 = nodes_df.loc[j, ["X", "Y"]]
                distance_matrix[i, j] = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

    # Step 3: Combine and prepare data
    nodes_df = nodes_df.merge(demands_df, on="Node")
    return {
        "nodes": nodes_df,
        "distance_matrix": distance_matrix,
        "capacity": capacity,
        "depot": depot
    }



# Inital feasible routes

In [32]:
def create_initial_feasible_solution_with_nodes(data):
    depot = data["depot"]  # Depot node (1 in this case)
    capacity = data["capacity"]  # Vehicle capacity
    nodes = data["nodes"]  # Node DataFrame with 'demand' and other details
    distance_matrix = data["distance_matrix"]  # Distance matrix
    customer_nodes = list(data["nodes"]["Node"])  # Extract all node numbers

    # Exclude the depot and segregate into odd and even customer nodes
    odd_customers = [node for node in customer_nodes if node != depot and node % 2 != 0]
    even_customers = [node for node in customer_nodes if node != depot and node % 2 == 0]
    

    # Create initial routes for each customer set
    def generate_routes(customer_set, Vehichle_type):
        initial_routes = {}
        for customer in customer_set:
            # route_id = f"Route_{customer}_{route_type}"  # Unique route ID
            route_id = f"Route_{customer}"  # Unique route ID
            route_cost = (
                2 * distance_matrix[depot - 1][customer - 1]
            )  # Depot -> Customer -> Depot
            # route_demand = nodes.loc[customer -1, "Demand"]  # Demand of the customer
            initial_routes[route_id] = {
                "cost": route_cost,
                "route": [depot, customer, depot],  # Depot -> Customer -> Depot
                "Type" : Vehichle_type,
            }
        return initial_routes

    # Generate routes for odd and even customers
    initial_routes_typeI = generate_routes(odd_customers, "TypeI")
    initial_routes_typeII = generate_routes(even_customers, "TypeII")

    # Combine all routes
    initial_routes = {**initial_routes_typeI, **initial_routes_typeII}

    return initial_routes

# Create vehicle types

In [33]:
def create_vehicle_types(data):
    """
    Create a dictionary of vehicle types with capacities based on demand.

    Args:
        data: Dictionary containing 'nodes' and 'capacity'.

    Returns:
        vehicle_types: Dictionary defining capacities for each vehicle type.
    """
    # Extract demands and node information
    nodes = data["nodes"]
    total_capacity = data["capacity"]

    # Segregate odd and even customers
    odd_demands = nodes[nodes["Node"] % 2 != 0]["Demand"]
    even_demands = nodes[nodes["Node"] % 2 == 0]["Demand"]

    # Calculate max demand for odd and even customers
    max_demand_odd = odd_demands.max()
    max_demand_even = even_demands.max()

    # Define vehicle capacities
    vehicle_types = {
        "TypeI": {
            "odd_capacity": max_demand_odd,
            "even_capacity": total_capacity - max_demand_odd
        },
        "TypeII": {
            "odd_capacity": total_capacity - max_demand_even,
            "even_capacity": max_demand_even
            
        }
    }

    return vehicle_types

# Create customers dictionary

In [34]:
def create_customers_dict(data):
    """
    Create a dictionary of customers with type and demand.

    Args:
        data: Dictionary containing 'nodes'.

    Returns:
        customers: Dictionary with customer details.
    """
    nodes = data["nodes"]

    customers = {
        row["Node"]: {
            "type": "odd" if row["Node"] % 2 != 0 else "even",
            "demand": row["Demand"]
        }
        for _, row in nodes.iterrows() if row["Node"] != data["depot"]
    }

    return customers


# Initial feasible solution

In [35]:
def initialize_and_solve_master_problem(routes, data, vehicle_types, customers, a_ir, route_costs):
    """
    Initializes and solves the Master Problem for the VRP using Gurobi.
    
    Args:
        routes (dict): Dictionary containing route information with the structure:
                       {
                           route_id: {
                               "Type": route_type,
                               "cost": route_cost,
                               "route": [list_of_customers]
                           },
                           ...
                       }
        data (dict): Dictionary containing problem data, specifically `nodes` which
                     includes the customers and depot information.
        vehicle_types (list): List of vehicle types (e.g., ["TypeI", "TypeII"]).
    
    Returns:
        tuple: A tuple containing:
               - solution (dict): Includes:
                   - 'Objective': Objective value of the solution.
                   - 'Variables': Selected route variables and their values.
                   - 'Status': Optimization status (e.g., Optimal, Infeasible).
               - dual_prices (dict): Dictionary of dual prices in the format {customer: dual_price}.
    """
    # Extract customers excluding the depot (Node 1)
    for route_id, route_info in routes.items():
        vehicle_type = route_info["Type"]
        route_costs[(route_id, vehicle_type)] = route_info["cost"]
        a_ir[vehicle_type][route_id] = {
            i: 1 if i in route_info["route"] else 0 for i in customers
        }

    # Initialize the Gurobi model
    master = gp.Model("MasterProblem")
    master.setParam('OutputFlag', 0)
    # Add decision variables for route selection
    lambda_vars = {}
    for route_id, route_info in routes.items():
        vehicle_type = route_info["Type"]
        lambda_vars[(route_id, vehicle_type)] = master.addVar(
            obj=route_info["cost"], vtype=GRB.CONTINUOUS,
            name=f"lambda_{route_id}_{vehicle_type}", lb=0, ub=1
        )

    # Add customer coverage constraints
    customer_constraints = {}
    for customer in customers:
        customer_constraints[customer] = master.addConstr(
            gp.quicksum(
                lambda_vars[(route_id, vehicle_type)] * a_ir[vehicle_type][route_id][customer]
                for route_id, route_info in routes.items()
                for vehicle_type in vehicle_types
                if vehicle_type == route_info["Type"]
            ) == 1,
            name=f"{customer}"
        )

    # Optimize the Master Problem
    master.optimize()

    # Initialize the solution dictionary
    solution = {}
    dual_prices = {}

    if master.Status == GRB.OPTIMAL:
        # Store the objective value
        solution["Objective"] = master.ObjVal

        # Store the variables with their values
        solution["Variables"] = {
            var.VarName: var.X for var in master.getVars() if var.X > 0
        }

        # Store the dual prices for each customer constraint
        for constr in master.getConstrs():
            dual_prices[int(constr.ConstrName)] = constr.Pi

        # Set the status
        solution["Status"] = "Optimal"
    else:
        # If the model isn't optimal, return empty values
        solution["Status"] = "Infeasible or Suboptimal"
        solution["Objective"] = None
        solution["Variables"] = {}

    # Return the solution dictionary and dual prices
    return solution, dual_prices, master, lambda_vars


# Utility functions

In [ ]:


def create_pricing_problem_graph_multi_vehicle(customers, arcs, dual_prices, vehicle_types):
    """
    Create directed graphs for the pricing problem with customer type-specific vehicle capacities.
    """
    graphs = {}
    
    for vehicle_name, vehicle_capacities in vehicle_types.items():
        # Create graph with multiple resource dimensions
        G = nx.DiGraph(directed=True, n_res=3)  # 4 resources
        
        for (i, j), costs in arcs.items():
            # Resource cost tracking:
            # 1st dimension: total number of customers visited
            # 2nd dimension: total demand for odd customers
            # 3rd dimension: total demand for even customers
            
            # Determine customer type and demand
            j_type = customers.get(j, {}).get('type', None)
            
            # Get demand, default to 0 if not a customer node
            j_demand = customers.get(j, {}).get('demand', 0)
            
            # Initialize resource cost array
            res_cost = np.array([
                1,  # customers visited
                j_demand if j_type == 'odd' else 0,  # odd customer demand
                j_demand if j_type == 'even' else 0  # even customer demand
            ])
            
            # Adjust cost by dual prices
            adjusted_cost = costs - dual_prices.get(i, 0)
            
            # Add edge to graph
            G.add_edge(i, j, res_cost=res_cost, weight=adjusted_cost)
        
        graphs[vehicle_name] = G
    
    return graphs

# Function to solve for each vehicle type
def solve_pricing_problem(graphs, customers, vehicle_types):
    results = {}
    time_limit = 120
    time_elapsed = 0
    for vehicle_name, vehicle_capacities in vehicle_types.items():
        # print("a")
        if time_elapsed > time_limit:
            continue
        G = graphs[vehicle_name]
        # Set resource constraints
        max_res = [
            len(G.edges()),  # max customers visited
            vehicle_capacities['odd_capacity'],  # max odd customer demand
            vehicle_capacities['even_capacity']  # max even customer demand
        ]
        # print(max_res)
        min_res = [0, 0, 0]
        
        # Run bidirectional algorithm
        try:
            time_start = time.time()
            bidirec = BiDirectional(G, max_res, min_res, direction="both", elementary=True, time_limit=5)
            bidirec.run()
            time_end = time.time()
            time_elapsed = time_end - time_start
            results[vehicle_name] = {
                'optimal_path': bidirec.path,
                'total_cost': bidirec.total_cost,
                'odd_capacity': vehicle_capacities['odd_capacity'],
                'even_capacity': vehicle_capacities['even_capacity'],
                'resources_used': bidirec.consumed_resources
            }
        except Exception as e:
            results[vehicle_name] = {
                'error': str(e)
            }
    
    return results


def compute_route_cost(path, depot, distance_matrix):
    """
    Compute the cost of a new route based on the path.
    """
    cost = 0
    for i in range(len(path) - 1):
        if path[i] == "Source":
            cost += distance_matrix[depot - 1][path[i + 1] - 1]
        elif path[i + 1] == "Sink":
            cost += distance_matrix[path[i] - 1][depot - 1]
        else:
            cost += distance_matrix[path[i] - 1][path[i + 1] - 1]
    return cost


def add_new_route_to_master(master, lambda_vars, routes, a_ir, new_route_id, new_route_type, new_route_cost, covered_customers, vehicle_types, customers):
    """
    Add a new route to the master problem.
    """
    # Update routes and a_ir
    a_ir[new_route_type][new_route_id] = {i: 1 if i in covered_customers else 0 for i in customers}
    routes[new_route_id] = {
        "cost": new_route_cost,
        "route": covered_customers,
        "Type": new_route_type,
    }

    # Add new lambda variable
    lambda_vars[(new_route_id, new_route_type)] = master.addVar(
        obj=new_route_cost, vtype=GRB.CONTINUOUS, name=f"lambda_{new_route_id}_{new_route_type}", lb=0, ub=1
    )

    master.update()

    return master


def rebuild_customer_constraints(master, lambda_vars, a_ir, routes, vehicle_types, customers):
    """
    Rebuild customer coverage constraints for the master problem.
    """
    # Remove existing constraints
    for constr in master.getConstrs():
        master.remove(constr)
    customer_constraints = {}
    for customer in customers:
        customer_constraints[customer] = master.addConstr(
            gp.quicksum(
                lambda_vars[(route_id, route_type)] * a_ir[route_type][route_id][customer]
                for route_id, route_info in routes.items()
                for route_type in vehicle_types
                if route_type == route_info["Type"]
            ) == 1,
            name=f"{customer}",
        )
    master.update()
    return master


def solve_master_problem(master):
    """
    Solve the master problem and return the optimal value and variable assignments.
    """
    master.setParam('OutputFlag', 0)
    master.optimize()
    if master.Status == GRB.OPTIMAL:
        master.write("master.lp")
        return master.objVal, {var.VarName: var.X for var in master.getVars()}
        
    else:
        raise Exception(f"Master problem optimization failed with status {master.Status}")


def process_fractional_solution(master, model_path="master.lp"):
    """
    Process fractional solutions by converting the master problem to a MIP.
    """
    # master.write(model_path)
    model = gp.read(model_path)
    model.setParam('OutputFlag', 0)
    model.setParam("MIPFocus", 1)     # Prioritize feasible solutions
    # model.setParam("MIPGap", 1e-4)
    # model.setParam("FeasibilityTol", 1e-6)
    # model.setParam("IntFeasTol", 1e-6)
    # Change variables to binary
    for var in model.getVars():
        var.vtype = GRB.BINARY
    model.update()
    model.optimize()
    gap = model.MIPGap
    for var in master.getVars():
    # if variable is fractional, then get the value from the MIP model
        if var.X < 1- 1e-4 and var.X > 0 + 1e-4:
            for var_mip in model.getVars():
                if var.VarName == var_mip.VarName:
                    var_mip.lb = var_mip.X
                # var_mip.ub = GRB.INFINITY
    model.write("model.lp")

    for var in model.getVars():
        var.vtype = GRB.CONTINUOUS
    model.update()
    model.optimize()

    return model, gap


def extract_dual_prices(model):
    """
    Extract dual prices from the model.
    """
    return {int(constr.ConstrName): constr.Pi for constr in model.getConstrs()}


# Pricing Problem

In [ ]:

def build_arcs_and_demands(customers, max_customers, depot, distance_matrix, nodes):
    """
    Build arcs and demands for a given set of customers.
    """
    arcs = {}

    for i in customers:
        if i < max_customers:
            arcs[("Source", i)] = distance_matrix[depot - 1][i - 1]
            arcs[(i, "Sink")] = distance_matrix[i - 1][depot - 1]

    for i in customers:
        for j in customers:
            if i != j and i < max_customers and j < max_customers:
                arcs[(i, j)] = distance_matrix[i - 1][j - 1]
                arcs[(j, i)] = distance_matrix[j - 1][i - 1]


    return arcs


def process_customer_set(
    max_customers,
    depot,
    distance_matrix,
    nodes,
    dual_prices,
    capacity,
    routes,
    lambda_vars,
    vehicle_types,
    master,
    a_ir,
    customers,
):
    """
    Process a set of customers (odd or even) by building the pricing problem,
    solving it, and adding the resulting route to the master problem.
    """
    start_pricing = time.time()

    arcs = build_arcs_and_demands(
        customers, max_customers, depot, distance_matrix, nodes
    )

    # Solve the pricing problem
    graphs = create_pricing_problem_graph_multi_vehicle(
        customers, arcs, dual_prices, vehicle_types)
    results = solve_pricing_problem(graphs, customers, vehicle_types)
            
    # Store the cost of the last added route to compare with the next one
    last_route_cost = None

    for vehicle_name, result in results.items():
        # print(result)
        if result['total_cost'] <= -1e-6:
            # If the cost of the current route matches the last route's cost, skip it
            if last_route_cost is not None and abs(result['total_cost'] - last_route_cost) <= 1e-6:
                continue

            # Compute route details
            new_route_cost = compute_route_cost(result["optimal_path"], depot, distance_matrix)
            covered_customers = [
                node for node in result["optimal_path"] if node != "Source" and node != "Sink"
            ]
            new_route_type = vehicle_name
            new_route_id = f"Extra_Route{len(lambda_vars) + 2}"

            # Add the new route to the master problem
            master = add_new_route_to_master(
                master,
                lambda_vars,
                routes,
                a_ir,
                new_route_id,
                new_route_type,
                new_route_cost,
                covered_customers,
                vehicle_types,
                customers,
            )

            # Update the last route cost
            last_route_cost = result['total_cost']

    pricing_time = time.time() - start_pricing

    
    return master, pricing_time

# Master Problem

In [38]:
def solve_and_update_master_problem(master, lambda_vars, a_ir, routes, vehicle_types, customers):
    """
    Rebuild customer constraints and solve the master problem.
    """
    start_master = time.time()

    master = rebuild_customer_constraints(master, lambda_vars, a_ir, routes, vehicle_types, customers)
    obj_val, variable_assignments = solve_master_problem(master)

    master_time = time.time() - start_master
    # print(f"Master Problem Objective Value = {obj_val}")
    return master, obj_val, variable_assignments, master_time

# Column Generation

In [39]:
def run_column_generation(
    master,
    # odd_customers,
    # even_customers,
    depot,
    distance_matrix,
    nodes,
    dual_prices,
    capacity,
    a_ir,
    routes,
    vehicle_types,
    customers,
    lambda_vars,
    prev_sol,
    increment,
    time_limit,
):
    """
    Run the column generation process for a fixed time limit.
    """
    start_time = time.time()
    iteration = 0
    max_customers = increment + 25
    max_customers = min(max_customers, len(customers))
    # max_customers = len(customers) + 2
    total_pricing_time = 0
    total_master_time = 0

    while time.time() - start_time < time_limit:
        iteration += 1

        
        master, pricing_time_odd = process_customer_set(
            max_customers,
            depot,
            distance_matrix,
            nodes,
            dual_prices,
            capacity,
            routes,
            lambda_vars,
            vehicle_types,
            master,
            a_ir,
            customers,
        )
        total_pricing_time += pricing_time_odd
       

        # Solve and update master problem
        master, obj_val, variable_assignments, master_time = solve_and_update_master_problem(
            master, lambda_vars, a_ir, routes, vehicle_types, customers
        )
        total_master_time += master_time

        # Process fractional solutions in master problem
        mip_model, gap = process_fractional_solution(master)
        dual_prices = extract_dual_prices(mip_model)


        if max_customers ==len(customers):
            prev_sol = obj_val
        # elif obj_val >= prev_sol:  
        #     max_customers += increment
        #     prev_sol = obj_val
        else:
            max_customers += increment
            prev_sol = obj_val
        # print(f"max_customers: {max_customers}")    


    # Output final results
    final_obj_val = mip_model.ObjVal
    final_gap = gap

    print(f"Final Objective Value: {final_obj_val}")
    print(f"Total Pricing Time: {total_pricing_time:.2f} seconds")
    print(f"Total Master Time: {total_master_time:.2f} seconds")
    print(f"Final MIP Gap: {final_gap}")

    return final_obj_val, total_pricing_time, total_master_time, final_gap

# Single Instance

In [40]:
def process_instance(file_path):
    """
    Process a single CVRP instance file and return results for the instance.
    """
    # Preprocess the data and initialize the problem
    start_time = time.time()
    
    data = preprocess_cvrp(file_path)
    routes = create_initial_feasible_solution_with_nodes(data)
    vehicle_types = create_vehicle_types(data)
    customers = create_customers_dict(data)

    # Precompute feasibility matrix a_ir and route costs
    a_ir = {t: {} for t in vehicle_types}
    route_costs = {}
    solution, dual_prices, master, lambda_vars = initialize_and_solve_master_problem(
        routes, data, vehicle_types, customers, a_ir, route_costs
    )

    # print(f"dual_prices: {dual_prices}")
    print(f"initial_solution: {solution['Objective']}")
    prev_sol = solution["Objective"]
    # Extract problem-specific parameters
    depot = data["depot"]
    capacity = data["capacity"]
    nodes = data["nodes"]
    distance_matrix = data["distance_matrix"]


    end_time = time.time()
    # Set column generation time limit
    time_limit = 300  - (end_time - start_time)

    # Run column generation and retrieve results
    final_obj_val, total_pricing_time, total_master_time, final_gap = run_column_generation(
        master,
        depot,
        distance_matrix,
        nodes,
        dual_prices,
        capacity,
        a_ir,
        routes,
        vehicle_types,
        customers,
        lambda_vars,
        prev_sol,
        increment = 2,
        time_limit=time_limit,
    )

    # Prepare results dictionary
    result = {
        "Instance": os.path.basename(file_path),  # Use the file name as the instance name
        "Root Node LP Objective": solution["Objective"],
        "Best PnB IP Solution": final_obj_val,
        "GAP": final_gap,
        "Time for Column Generation": total_pricing_time,
        "Time for IP": total_master_time,
    }
    return result

# Main

In [41]:

def main():
    """
    Main function to process all VRP instance files in the specified directory
    and return a consolidated DataFrame of results.
    """
    results = []  # Initialize an empty list to store results

    for root, dirs, files in os.walk("./Uchoa-Vrp-Set-X"):
        files = sorted(files)
        for file in files:
            if file.endswith(".vrp"):  # Process only .vrp files
                print(f"----------------{file}----------------")
                file_path = os.path.join(root, file)
                try:
                    result = process_instance(file_path)
                    results.append(result)
                except Exception as e:
                    print(f"Error processing {file}: {e}")

    # Convert results to a Pandas DataFrame
    results_df = pd.DataFrame(results)

    # Save the results to a CSV file (optional)
    results_df.to_csv("cvrp_results_scenario_2.csv", index=False)

    return results_df




In [42]:
if __name__ == "__main__":
    results_df = main()
    print(results_df)


----------------X-n101-k25.vrp----------------
initial_solution: 90010.73456916604
Read LP format model from file master.lp
Reading time = 0.00 seconds
: 100 rows, 102 columns, 111 nonzeros
Read LP format model from file master.lp
Reading time = 0.00 seconds
: 100 rows, 103 columns, 117 nonzeros
Read LP format model from file master.lp
Reading time = 0.00 seconds
: 100 rows, 105 columns, 128 nonzeros
Read LP format model from file master.lp
Reading time = 0.00 seconds
: 100 rows, 106 columns, 134 nonzeros
Read LP format model from file master.lp
Reading time = 0.00 seconds
: 100 rows, 107 columns, 140 nonzeros
Read LP format model from file master.lp
Reading time = 0.00 seconds
: 100 rows, 109 columns, 147 nonzeros
Read LP format model from file master.lp
Reading time = 0.00 seconds
: 100 rows, 110 columns, 151 nonzeros
Read LP format model from file master.lp
Reading time = 0.00 seconds
: 100 rows, 112 columns, 159 nonzeros
Read LP format model from file master.lp
Reading time = 0.00 